#  基于MindSpore实现LSTM算法 
本实验基于MindSpore构建LSTM模型，使用SentimentNet实现情感分类,并训练和测试模型。
## 1 实验目的
1.通过实验了解LSTM算法

2.基于MindSpore中实现LSTM算法
## 2 LSTM算法原理介绍
LSTM四个函数层与具体介绍如下：

(1)第一个函数层：遗忘门
    ![jupyter](./Figures/fig001.png)
 
对于上一时刻LSTM中的单元状态，一些信息可能会随着时间的流逝而过时。为了不让过多记忆影响神经网络对现在输入的处理，我们应该选择性遗忘一些在之前单元状态中的分量——这个工作就交给了遗忘门。

每一次输入一个新的输入，LSTM会先根据新的输入和上一时刻的输出决定遗忘之前的哪些记忆——输入和上一步的输出会整合为一个单独的向量，然后通过sigmoid神经层，最后点对点的乘在单元状态上。因为sigmoid 函数会将任意输入压缩到 (0,1) 的区间上，我们可以非常直觉的得出这个门的工作原理 —— 如果整合后的向量某个分量在通过sigmoid层后变为0，那么显然单元状态在对位相乘后对应的分量也会变成0，换句话说，遗忘了这个分量上的信息；如果某个分量通过sigmoid层后为1，单元状态会“保持完整记忆”。不同的sigmoid输出会带来不同信息的记忆与遗忘。通过这种方式，LSTM可以长期记忆重要信息，并且记忆可以随着输入进行动态调整。下面的公式可以用来描述遗忘门的计算，其中f_t就是sigmoid神经层的输出向量：

$f_t=σ(W_f∙[h_(t-1),x_t ]+b_f)$

（2）第二个、第三个函数层：记忆门
记忆门是用来控制是否将在t时刻（现在）的数据并入单元状态中的控制单位。首先，用tanh函数层将现在的向量中的有效信息提取出来，然后使用sigmoid函数来控制这些记忆要放多少进入单元状态。这两者结合起来就可以做到：

 ![jupyter](./Figures/fig002.png)
 
从当前输入中提取有效信息；对提取的有效信息做出筛选，为每个分量做出评级(0 ~ 1)，评级越高的最后会有越多的记忆进入单元状态。下面的公式可以分别表示这两个步骤在LSTM中的计算：

$C_{t}^{'}=tanh⁡(W_c∙[h_{(t-1)},x_t ]+b_c)$

$i_t=σ(W_i∙[h_{(t-1)},x_t ]+b_i)$

（3）第四个函数层：输出门

输出门就是LSTM单元用于计算当前时刻的输出值的神经层。输出层会先将当前输入值与上一时刻输出值整合后的向量用sigmoid函数提取其中的信息，然后，会将当前的单元状态通过tanh函数压缩映射到区间(-1, 1)中，将经过tanh函数处理后的单元状态与sigmoid函数处理后的单元状态，整合后的向量点对点的乘起来就可以得到LSTM在 t时刻的输出。

LSTM模型是由时刻的输入词$X_{t}$ ，细胞状态$C_{t}$，临时细胞状态$\widetilde{C_{t}} $，隐层状态$h_{t}$，遗忘门$f_{t}$，记忆门$i_{t}$，输出门$ o_{t}$组成。LSTM的计算过程可以概括为:通过对细胞状态中信息遗忘和记忆新的信息使得对后续时刻计算有用的信息得以传递，而无用的信息被丢弃，并在每个时间步都会输出隐层状态$h_{t}$ ，其中遗忘、记忆与输出由通过上个时刻的隐层状态$h_{t-1}$和当前输入$X_{t}$计算出来的遗忘门$f_{t}$，记忆门$ i_{t}$，输出门$o_{t}$来控制。

LSTM总体框架图如下：
 ![jupyter](./Figures/fig003.png)


## 3 实验环境
### 实验环境要求
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|
## 4 数据处理
IMDB是一个与国内豆瓣比较类似的与电影相关的网站，而本次实验用到的数据集是这个网站中的一些用户评论。IMDB数据集共包含50000项影评文字，训练数据和测试数据各25000项，每一项影评文字都被标记为正面评价或负面评价，所以本实验可以看做一个二分类问题。IMDB数据集官网：[Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/)。

方式一，从斯坦福大学官网下载aclImdb_v1.tar.gz并解压。

方式二，从华为云OBS中下载aclImdb_v1.tar.gz并解压。

同时，我们要下载GloVe文件，并在文件glove.6B.300d.txt开头处添加新的一行400000 300，即总共读取400000个单词，每个单词用300维度的词向量表示。 修改glove.6B.300.txt如下:

400000 300

the -0.071549 0.093459 0.023738 -0.090339 0.056123 0.32547…

确定评价标准：
作为典型的分类问题，情感分类的评价标准可以比照普通的分类问题处理。常见的精度（Accuracy）、精准度（Precision）、召回率（Recall）和F_beta分数都可以作为参考。

精度（Accuracy）=分类正确的样本数目/总样本数目

精准度（Precision）=真阳性样本数目/所有预测类别为阳性的样本数目

召回率（Recall）=真阳性样本数目/所有真实类别为阳性的样本数目

F1分数=(2∗Precision∗Recall)/(Precision+Recall)

在IMDB这个数据集中，正负样本数差别不大，可以简单地用精度（accuracy）作为分类器的衡量标准。

## 5 模型构建
（1）导入Python库&模块并配置运行信息

导入MindSpore模块和辅助模块。

代码如下：

In [1]:
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore.train import Model
from mindspore import context

(2）定义参数变量

device_target：指定Ascend或CPU/GPU环境。
pre_trained：预加载CheckPoint文件。
preprocess：是否预处理数据集，默认为否。
aclimdb_path：数据集存放路径。
glove_path：GloVe文件存放路径。
preprocess_path：预处理数据集的结果文件夹。
ckpt_path：CheckPoint文件路径。   

注意：若在GPU上训练需要更改 'device_target': "CPU"为 'device_target': "GPU"。

In [2]:
#设置参数变量
from easydict import EasyDict as edict
lstm_cfg = edict({
    'num_classes': 2,
    'learning_rate': 0.1,
    'momentum': 0.9,
    'num_epochs': 1,    
    'batch_size': 128,
    'embed_size': 300,
    'num_hiddens': 100,
    'num_layers': 2,
    'bidirectional': True,
    'save_checkpoint_steps': 390,
    'keep_checkpoint_max': 10
})
args_train = edict({
    'preprocess': 'true',
    'aclimdb_path': "./aclImdb",
    'glove_path': "./glove",
    'preprocess_path': "./preprocess",
    'ckpt_path': "./",
    'pre_trained': None,
    'device_target': "CPU",
})
args_test = edict({
    'preprocess': 'false',
    'aclimdb_path': "./aclImdb",
    'glove_path': "./glove",
    'preprocess_path': "./preprocess",
    'ckpt_path': "./lstm-1_195.ckpt",
    'pre_trained': None,
    'device_target': "CPU",
})

（3）数据的读取与处理

按照下面的流程解析原始数据集，获得features与labels：sentence->tokenized->encoded->padding->features。

定义convert_to_mindrecord函数将数据集格式转换为MindRecord格式，便于MindSpore读取。
对文本数据集进行处理，包括编码、分词、对齐、处理GloVe原始数据，使之能够适应网络结构。

定义convert_to_mindrecord函数将数据集格式转换为MindRecord格式，便于MindSpore读取。

转换成功后会在`preprocess`目录下生成MindRecord文件，通常该操作在数据集不变的情况下，无需每次训练都执行，此时查看`preprocess`文件目录结构。

```text
preprocess
├── aclImdb_test.mindrecord0
├── aclImdb_test.mindrecord0.db
├── aclImdb_test.mindrecord1
├── aclImdb_test.mindrecord1.db
├── aclImdb_test.mindrecord2
├── aclImdb_test.mindrecord2.db
├── aclImdb_test.mindrecord3
├── aclImdb_test.mindrecord3.db
├── aclImdb_train.mindrecord0
├── aclImdb_train.mindrecord0.db
├── aclImdb_train.mindrecord1
├── aclImdb_train.mindrecord1.db
├── aclImdb_train.mindrecord2
├── aclImdb_train.mindrecord2.db
├── aclImdb_train.mindrecord3
├── aclImdb_train.mindrecord3.db
└── weight.txt
```

In [3]:
import os
from itertools import chain
import numpy as np
import gensim
class ImdbParser():
    def __init__(self, imdb_path, glove_path, embed_size=300):
        self.__segs = ['train', 'test']
        self.__label_dic = {'pos': 1, 'neg': 0}
        self.__imdb_path = imdb_path
        self.__glove_dim = embed_size
        self.__glove_file = os.path.join(glove_path, 'glove.6B.' + str(self.__glove_dim) + 'd.txt')
        self.__imdb_datas = {}
        self.__features = {}
        self.__labels = {}
        self.__vacab = {}
        self.__word2idx = {}
        self.__weight_np = {}
        self.__wvmodel = None
#解析IMDB数据集，生成特征、标签和权重矩阵
    def parse(self):
        self.__wvmodel = gensim.models.KeyedVectors.load_word2vec_format(self.__glove_file)
        for seg in self.__segs:
# 解析IMDB数据
            self.__parse_imdb_datas(seg)
# 解析特征和标签
            self.__parse_features_and_labels(seg)
# 生成权重数组
            self.__gen_weight_np(seg)
#解析IMDB数据文件，获取文本和标签对应的列表
    def __parse_imdb_datas(self, seg):
        data_lists = []
        for label_name, label_id in self.__label_dic.items():
            sentence_dir = os.path.join(self.__imdb_path, seg, label_name)
            for file in os.listdir(sentence_dir):
                with open(os.path.join(sentence_dir, file), mode='r', encoding='utf8') as f:
                    sentence = f.read().replace('\n', '')
                    data_lists.append([sentence, label_id])
        self.__imdb_datas[seg] = data_lists
    def __parse_features_and_labels(self, seg):
        features = []
        labels = []
        for sentence, label in self.__imdb_datas[seg]:
            features.append(sentence)
            labels.append(label)
        self.__features[seg] = features
        self.__labels[seg] = labels
# 更新特征为令牌化形式
        self.__updata_features_to_tokenized(seg)
# 解析词汇表
        self.__parse_vacab(seg)
# 编码特征
        self.__encode_features(seg)
# 填充特征
        self.__padding_features(seg)
#将特征（文本）转换为令牌化形式
    def __updata_features_to_tokenized(self, seg):
        tokenized_features = []
        for sentence in self.__features[seg]:
            tokenized_sentence = [word.lower() for word in sentence.split(" ")]
            tokenized_features.append(tokenized_sentence)
        self.__features[seg] = tokenized_features
#解析词汇表，生成词汇表和单词到索引的映射
    def __parse_vacab(self, seg):
        tokenized_features = self.__features[seg]
        vocab = set(chain(*tokenized_features))
        self.__vacab[seg] = vocab
        word_to_idx = {word: i + 1 for i, word in enumerate(vocab)}
        word_to_idx['<unk>'] = 0
        self.__word2idx[seg] = word_to_idx
#将特征（令牌化形式）编码为索引序列
    def __encode_features(self, seg):
        word_to_idx = self.__word2idx['train']
        encoded_features = []
        for tokenized_sentence in self.__features[seg]:
            encoded_sentence = []
            for word in tokenized_sentence:
                encoded_sentence.append(word_to_idx.get(word, 0))
            encoded_features.append(encoded_sentence)
        self.__features[seg] = encoded_features
#填充特征序列，使其具有相同的长度
    def __padding_features(self, seg, maxlen=500, pad=0):
        padded_features = []
        for feature in self.__features[seg]:
            if len(feature) >= maxlen:
                padded_feature = feature[:maxlen]
            else:
                padded_feature = feature
                while len(padded_feature) < maxlen:
                    padded_feature.append(pad)
            padded_features.append(padded_feature)
        self.__features[seg] = padded_features
#生成权重矩阵，用于将词汇转换为预训练的词向量
    def __gen_weight_np(self, seg):
        weight_np = np.zeros((len(self.__word2idx[seg]), self.__glove_dim), dtype=np.float32)
        for word, idx in self.__word2idx[seg].items():
            if word not in self.__wvmodel:
                continue
            word_vector = self.__wvmodel.get_vector(word)
            weight_np[idx, :] = word_vector
        self.__weight_np[seg] = weight_np
#获取指定数据集的特征、标签和权重矩阵
    def get_datas(self, seg):
        features = np.array(self.__features[seg]).astype(np.int32)
        labels = np.array(self.__labels[seg]).astype(np.int32)
        weight = np.array(self.__weight_np[seg])
        return features, labels, weight

In [7]:
import os
import numpy as np
import mindspore.dataset as ds
from mindspore.mindrecord import FileWriter
#创建MindSpore数据集
def lstm_create_dataset(data_home, batch_size, repeat_num=1, training=True):
    ds.config.set_seed(1)
    data_dir = os.path.join(data_home, "aclImdb_train.mindrecord0")
    if not training:
        data_dir = os.path.join(data_home, "aclImdb_test.mindrecord0")
# 从MindRecord文件中读取数据集
    data_set = ds.MindDataset(data_dir, columns_list=["feature", "label"], num_parallel_workers=4)
# 对数据集进行洗牌
    data_set = data_set.shuffle(buffer_size=data_set.get_dataset_size())
# 按批次进行数据集划分
    data_set = data_set.batch(batch_size=batch_size, drop_remainder=True)
# 对数据集进行重复
    data_set = data_set.repeat(count=repeat_num)
    return data_set
# 将特征和标签转换为MindRecord文件格式
def _convert_to_mindrecord(data_home, features, labels, weight_np=None, training=True):
    if weight_np is not None:
        np.savetxt(os.path.join(data_home, 'weight.txt'), weight_np)
    schema_json = {"id": {"type": "int32"},
                   "label": {"type": "int32"},
                   "feature": {"type": "int32", "shape": [-1]}}
    data_dir = os.path.join(data_home, "aclImdb_train.mindrecord")
    if not training:
        data_dir = os.path.join(data_home, "aclImdb_test.mindrecord")
    def get_imdb_data(features, labels):
        data_list = []
        for i, (label, feature) in enumerate(zip(labels, features)):
            data_json = {"id": i,
                         "label": int(label),
                         "feature": feature.reshape(-1)}
            data_list.append(data_json)
        return data_list
    writer = FileWriter(data_dir, shard_num=4)
# 将数据转换为MindRecord的数据格式
    data = get_imdb_data(features, labels)
# 添加schema和索引
    writer.add_schema(schema_json, "nlp_schema")
    writer.add_index(["id", "label"])
    writer.write_raw_data(data)
    writer.commit()
# 将数据集转换为MindRecord文件格式
def convert_to_mindrecord(embed_size, aclimdb_path, preprocess_path, glove_path):
    parser = ImdbParser(aclimdb_path, glove_path, embed_size)
    parser.parse()
    if not os.path.exists(preprocess_path):
        print(f"preprocess path {preprocess_path} is not exist")
        os.makedirs(preprocess_path)
#获取训练集数据
    train_features, train_labels, train_weight_np = parser.get_datas('train')
    _convert_to_mindrecord(preprocess_path, train_features, train_labels, train_weight_np)
# 获取测试集数据
    test_features, test_labels, _ = parser.get_datas('test')
    _convert_to_mindrecord(preprocess_path, test_features, test_labels, training=False)

(4)模型构建

导入初始化网络所需模块。

定义需要单层LSTM小算子堆叠的设备类型。

定义lstm_default_state函数来初始化网络参数及网络状态。

定义stack_lstm_default_state函数来初始化小算子堆叠需要的初始化网络参数及网络状态。

针对CPU场景，自定义单层LSTM小算子堆叠，来实现多层LSTM大算子功能。

使用Cell方法，定义网络结构（SentimentNet网络）。

实例化SentimentNet，创建网络，最后输出网络中加载的参数。

In [8]:
import mindspore.nn as nn
import mindspore.ops as ops
class SentimentNet(nn.Cell):
    def __init__(self,
                 vocab_size,
                 embed_size,
                 num_hiddens,
                 num_layers,
                 bidirectional,
                 num_classes,
                 weight,
                 batch_size):
        super(SentimentNet, self).__init__()
# 创建嵌入层
        self.embedding = nn.Embedding(vocab_size,
                                      embed_size,
                                      embedding_table=weight)
        self.embedding.embedding_table.requires_grad = False
# 转置操作
        self.trans = ops.Transpose()
        self.perm = (1, 0, 2)
# 创建LSTM编码器
        self.encoder = nn.LSTM(input_size=embed_size,
                               hidden_size=num_hiddens,
                               num_layers=num_layers,
                               has_bias=True,
                               bidirectional=bidirectional,
                               dropout=0.0)
# 拼接操作
        self.concat = ops.Concat(1)
# 创建全连接层作为解码器
        if bidirectional:
            self.decoder = nn.Dense(num_hiddens * 4, num_classes)
        else:
            self.decoder = nn.Dense(num_hiddens * 2, num_classes)
    def construct(self, inputs):
        embeddings = self.embedding(inputs)
        embeddings = self.trans(embeddings, self.perm)
        output, _ = self.encoder(embeddings)
        encoding = self.concat((output[0], output[499]))
        outputs = self.decoder(encoding)
        return outputs

## 6 模型训练

在训练前，需要对数据进行预处理。

根据建立的LSTM模型，对模型进行训练：

运行以下一段代码，创建优化器和损失函数模型，加载训练数据集（ds_train）并配置好CheckPoint生成信息，然后使用model.train接口，进行模型训练。根据输出可以看到loss值随着训练逐步降低。

In [9]:
import argparse
import os
import numpy as np
from mindspore import Tensor, nn, context, load_param_into_net, load_checkpoint
from mindspore.train import LossMonitor, CheckpointConfig, ModelCheckpoint, TimeMonitor,Accuracy, Model
# 如果条件为真，则执行以下代码块
if 1:
    args = args_train
    cfg = lstm_cfg
# 设置运行模式和设备目标
    context.set_context(
        mode=context.GRAPH_MODE,
        save_graphs=False,
        device_target=args.device_target)

注意：存放预处理结果的目标文件夹如果已经生成了权重文件，以下代码不要重复运行

In [10]:
#数据预处理，生成权重文件
if args.preprocess == "true":
    print("============== Starting Data Pre-processing ==============")
    convert_to_mindrecord(cfg.embed_size, args.aclimdb_path, args.preprocess_path, args.glove_path)

============== Starting Data Pre-processing ==============


训练模型：

In [11]:
#加载嵌入矩阵
embedding_table = np.loadtxt(os.path.join(args.preprocess_path, "weight.txt")).astype(np.float32)
# 创建网络
network = SentimentNet(vocab_size=embedding_table.shape[0],
                           embed_size=cfg.embed_size,
                           num_hiddens=cfg.num_hiddens,
                           num_layers=cfg.num_layers,
                           bidirectional=cfg.bidirectional,
                           num_classes=cfg.num_classes,
                           weight=Tensor(embedding_table),
                           batch_size=cfg.batch_size)
# 加载预训练模型参数
if args.pre_trained:
    load_param_into_net(network, load_checkpoint(args.pre_trained))
# 定义损失函数、优化器和评价指标
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
opt = nn.Momentum(network.trainable_params(), cfg.learning_rate, cfg.momentum)
loss_cb = LossMonitor()
model = Model(network, loss, opt, {'acc': Accuracy()})
print("============== Starting Training ==============")
# 训练模型
num_steps = 10
ds_train = lstm_create_dataset(args.preprocess_path, cfg.batch_size, 1)
config_ck = CheckpointConfig(save_checkpoint_steps=cfg.save_checkpoint_steps,
                                 keep_checkpoint_max=cfg.keep_checkpoint_max)
ckpoint_cb = ModelCheckpoint(prefix="lstm", directory=args.ckpt_path, config=config_ck)
time_cb = TimeMonitor(data_size=ds_train.get_dataset_size())
# 根据设备目标选择训练方式
if args.device_target == "CPU":
    model.train(cfg.num_epochs, ds_train, callbacks=[time_cb, ckpoint_cb, loss_cb], dataset_sink_mode=False)
else:
    model.train(cfg.num_epochs, ds_train, callbacks=[time_cb, ckpoint_cb, loss_cb])
print("============== Training Success ==============")

============== Starting Training ==============
epoch: 1 step: 1, loss is 0.693895697593689
epoch: 1 step: 2, loss is 0.6929268836975098
epoch: 1 step: 3, loss is 0.6929802298545837
epoch: 1 step: 4, loss is 0.6938257813453674
epoch: 1 step: 5, loss is 0.6931001543998718
epoch: 1 step: 6, loss is 0.692486047744751
epoch: 1 step: 7, loss is 0.6938748359680176
epoch: 1 step: 8, loss is 0.6904892921447754
epoch: 1 step: 9, loss is 0.6954400539398193
epoch: 1 step: 10, loss is 0.6928492188453674
epoch: 1 step: 11, loss is 0.6947997808456421
epoch: 1 step: 12, loss is 0.6929847598075867
epoch: 1 step: 13, loss is 0.6927945017814636
epoch: 1 step: 14, loss is 0.6951846480369568
epoch: 1 step: 15, loss is 0.6923548579216003
epoch: 1 step: 16, loss is 0.6974267959594727
epoch: 1 step: 17, loss is 0.691119372844696
epoch: 1 step: 18, loss is 0.6984376311302185
epoch: 1 step: 19, loss is 0.6991745829582214
epoch: 1 step: 20, loss is 0.6967753171920776
epoch: 1 step: 21, loss is 0.700273156166076

epoch: 1 step: 178, loss is 0.6696520447731018
epoch: 1 step: 179, loss is 0.666671633720398
epoch: 1 step: 180, loss is 0.6843529343605042
epoch: 1 step: 181, loss is 0.6474733948707581
epoch: 1 step: 182, loss is 0.6565805673599243
epoch: 1 step: 183, loss is 0.660170316696167
epoch: 1 step: 184, loss is 0.648608922958374
epoch: 1 step: 185, loss is 0.6621960401535034
epoch: 1 step: 186, loss is 0.6408436894416809
epoch: 1 step: 187, loss is 0.6026341319084167
epoch: 1 step: 188, loss is 0.6355273723602295
epoch: 1 step: 189, loss is 0.5880937576293945
epoch: 1 step: 190, loss is 0.5902379155158997
epoch: 1 step: 191, loss is 0.6481162905693054
epoch: 1 step: 192, loss is 0.6000613570213318
epoch: 1 step: 193, loss is 0.5818831324577332
epoch: 1 step: 194, loss is 0.6269707679748535
epoch: 1 step: 195, loss is 0.6468162536621094
Train epoch time: 3061722.619 ms, per step time: 15701.142 ms
============== Training Success ==============


## 7 模型测试
根据处理后的测试数据以及建立的LSTM模型，对模型进行测试：

创建并加载验证数据集（ds_eval），加载由训练保存的CheckPoint文件，进行验证，查看模型质量。

In [17]:
import argparse
import os
import numpy as np
from mindspore import Tensor, nn, context, load_checkpoint, load_param_into_net
from mindspore.train import Accuracy
from mindspore.train import LossMonitor, Model
if 1:
    args = args_test
    context.set_context(
        mode=context.GRAPH_MODE,
        save_graphs=False,
        device_target=args.device_target)
    if args.preprocess == "true":
        print("============== Starting Data Pre-processing ==============")
        convert_to_mindrecord(cfg.embed_size, args.aclimdb_path, args.preprocess_path, args.glove_path)
    embedding_table = np.loadtxt(os.path.join(args.preprocess_path, "weight.txt")).astype(np.float32)
    network = SentimentNet(vocab_size=embedding_table.shape[0],
                           embed_size=cfg.embed_size,
                           num_hiddens=cfg.num_hiddens,
                           num_layers=cfg.num_layers,
                           bidirectional=cfg.bidirectional,
                           num_classes=cfg.num_classes,
                           weight=Tensor(embedding_table),
                           batch_size=cfg.batch_size)
# 定义损失函数和优化器
    loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
    opt = nn.Momentum(network.trainable_params(), cfg.learning_rate, cfg.momentum)
# 定义评估指标
    loss_cb = LossMonitor()
    model = Model(network, loss, opt, {'acc': Accuracy()})
    print("============== Starting Testing ==============")
# 创建评估数据集
    ds_eval = lstm_create_dataset(args.preprocess_path, cfg.batch_size, training=False)
    param_dict = load_checkpoint(args.ckpt_path)
# 加载训练好的模型参数
    load_param_into_net(network, param_dict)
# 在设备上进行评估
    if args.device_target == "CPU":
        acc = model.eval(ds_eval, dataset_sink_mode=False)
    else:
        acc = model.eval(ds_eval)
    print("============== {} ==============".format(acc))

============== Starting Testing ==============
============== {'acc': 0.6401842948717948} ==============


## 8 实验总结

以上便完成了MindSpore自然语言处理应用的体验，通过本次体验全面了解了如何使用MindSpore进行自然语言中处理情感分类问题，理解了如何通过定义和初始化基于LSTM的SentimentNet网络进行训练模型及验证正确率。